In [0]:
import numpy as np
import pandas as pd
from os import listdir
import keras


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
 colab					    dictionary.gsheet
'Copy of dictionary.gsheet'		    Dictionary.ipynb
'Copy of DK - Big ideas simply explained'   findapp.gsheet
 dataset				    sentence_classification
 data.txt				   'Untitled presentation.gslides'


In [0]:
b =['SearchCreativeWork','GetWeather','BookRestaurant','PlayMusic',
    'AddToPlaylist','RateBook','SearchScreeningEvent']

In [0]:
data=[]
for i in range(len(b)):
    path = '/content/drive/My Drive/sentence_classification/data/'+b[i]+'.txt'
    df = pd.read_csv(path,names=['sentence'],sep='\n')
    df['lables'] = i
    print('read',len(df),"file from",b[i],'.txt')
    data.append(df)
data = pd.concat(data)

read 1954 file from SearchCreativeWork .txt
read 2001 file from GetWeather .txt
read 1973 file from BookRestaurant .txt
read 2000 file from PlayMusic .txt
read 1943 file from AddToPlaylist .txt
read 1971 file from RateBook .txt
read 1960 file from SearchScreeningEvent .txt


In [0]:
labels = data['lables'].values
sentence = data['sentence'].values

In [0]:
labels = keras.utils.to_categorical(labels,len(b))

In [0]:
sentences_train,sentences_test,y_train,y_test = train_test_split(sentence,labels,test_size=0.25,random_state=13)

In [0]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(sentences_train[0])
print(X_train[0])

rate this book one for this stars judging
[23, 21, 11, 53, 5, 21, 38, 3748]


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[1])

[  28   16    1   54    6   19    1  619 3749 3750    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [0]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath,encoding="utf-8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [0]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('/content/drive/My Drive/sentence_classification/glove.6B.50d.txt', 
                                           tokenizer.word_index, embedding_dim)
from keras.models import Sequential

from keras import layers

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(7, activation='sigmoid'))


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
import functools

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1)
top1_acc.__name__ = 'top1_acc'

model.compile(optimizer='adam',
              loss="binary_crossentropy", #'binary_crossentropy',
              metrics=["accuracy",top3_acc])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           485700    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 128)           32128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 77        
Total params: 519,195
Trainable params: 33,495
Non-trainable params: 485,700
_________________________________________________________________


In [0]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 10351 samples, validate on 3451 samples
Epoch 1/5
10351/10351 [==============================] - 10s 940us/step - loss: 0.1186 - acc: 0.9577 - top3_acc: 0.9637 - val_loss: 0.0386 - val_acc: 0.9877 - val_top3_acc: 0.9962
Epoch 2/5
10351/10351 [==============================] - 7s 653us/step - loss: 0.0296 - acc: 0.9911 - top3_acc: 0.9987 - val_loss: 0.0269 - val_acc: 0.9904 - val_top3_acc: 0.9983
Epoch 3/5
10351/10351 [==============================] - 7s 656us/step - loss: 0.0173 - acc: 0.9945 - top3_acc: 0.9997 - val_loss: 0.0238 - val_acc: 0.9921 - val_top3_acc: 0.9988
Epoch 4/5
10351/10351 [==============================] - 7s 659us/step - loss: 0.0100 - acc: 0.9973 - top3_acc: 0.9998 - val_loss: 0.0258 - val_acc: 0.9911 - val_top3_acc: 0.9977
Epoch 5/5
10351/10351 [==============================] - 7s 648us/step - loss: 0.0065 - acc: 0.9981 - top3_a